# Multivariate Entropy

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../core/')
from te import bi_te, mv_te

In [2]:
def tsGen3(n=10000):
    """
    VAR(2) for 4 variates x1[t], x2[t], x3[t], x4[t]
    """
    np.random.seed(200)
    x=np.zeros((2*n,4))
    Veps = np.eye(4)
    e = np.random.multivariate_normal(np.zeros(4), cov=Veps,size=2*n)
    
    for i in range(2,2*n):
        x[i,0] = 0.8*x[i-1,0] + e[i,0]
        x[i,1] = 0.8*x[i-1,1] + 0.5*x[i-2,0] + e[i,1]
        x[i,2] = 0.7*x[i-1,0] + e[i,2]
        x[i,3] = 0.7*x[i-2,0] + e[i,3]
    return x[n:,:]  

In [3]:
x = tsGen3(n=5000)

## Transfer entropy

In [4]:
#Settings
lag = 3  #=embedding dimension, lag at which the TEs are computed
knn_ = int(np.sqrt(x.shape[0])/2)    #value of k in the KNN method used in KSG estimator - can be set manually

### Sanity check
Both bivariate and multivariate TE should return the same value of single source single target. 

In [5]:

temv_ = mv_te(X=[x[:,0]], y=x[:,1], embDim=lag).multiSrc_ksg(k=20)
tebi_ = bi_te(x[:,0], x[:,1], embDim=lag).ksg(k=20)
print(temv_,tebi_)

0.15053861116670353 0.15053861116670353


### TE estimate for multiple sources to a single target

In [6]:
te134_2 = mv_te(X=[x[:,0],x[:,2],x[:,3]], y=x[:,1], embDim=lag).multiSrc_ksg(k=knn_)
print(te134_2)

0.12177863590079818


In [7]:
te34_2 = mv_te(X=[x[:,2],x[:,3]], y=x[:,1], embDim=lag).multiSrc_ksg(k=knn_)
print(te34_2)

0.08320960816564771


In [8]:
te14_2 = mv_te(X=[x[:,0],x[:,3]], y=x[:,1], embDim=lag).multiSrc_ksg(k=knn_)
print(te14_2)

0.128547036060485


In [9]:
te13_2 = mv_te(X=[x[:,0],x[:,2]], y=x[:,1], embDim=lag).multiSrc_ksg(k=knn_)
print(te13_2)

0.13091874246195312


In [10]:
print('TE 1->2',te134_2-te34_2)
print('TE 3->2',te134_2-te14_2)
print('TE 4->2',te134_2-te13_2)

TE 1->2 0.038569027735150474
TE 3->2 -0.006768400159686827
TE 4->2 -0.009140106561154937


### Net TE from each source within multiple sources to a single target

Instead of manually computing the TEs in the above, we can use the following method from the same class:

In [11]:
teDict = mv_te(X=[x[:,0],x[:,2],x[:,3]], y=x[:,1], embDim=lag).ntwrk_ksg(k=knn_)
print(teDict)

{'TE_all->y': 0.12177863590079818, 'TE0->y': 0.038569027735150474, 'TE1->y': -0.006768400159686827, 'TE2->y': -0.009140106561154937}
